In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

In [2]:
df_train = pd.read_csv("test.csv")
df_train_copy = df_train.copy()

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Id                                           2500 non-null   object 
 1   User Name                                    2500 non-null   object 
 2   Personal URL                                 1038 non-null   object 
 3   Profile Cover Image Status                   2480 non-null   object 
 4   Profile Verification Status                  2500 non-null   object 
 5   Profile Text Color                           2464 non-null   object 
 6   Profile Page Color                           2473 non-null   object 
 7   Profile Theme Color                          2477 non-null   object 
 8   Is Profile View Size Customized?             2500 non-null   bool   
 9   UTC Offset                                   2352 non-null   float64
 10  

**Dropping classes**

In [4]:
df_train.drop(labels=["User Name", "Personal URL", "Location", "User Time Zone"], axis=1, inplace=True)

**Managing TimeStamp**

In [5]:
trips_start = df_train[["Id", "Profile Creation Timestamp"]].copy()
trips_start["Profile Creation Timestamp"] = pd.to_datetime(trips_start["Profile Creation Timestamp"])

#https://www.w3resource.com/python-exercises/pandas/datetime/pandas-datetime-exercise-8.php
trips_start["Profile Creation Day"] = trips_start["Profile Creation Timestamp"].dt.day
trips_start["Profile Creation WeekDay"] = trips_start["Profile Creation Timestamp"].dt.day_name()
trips_start["Profile Creation Month"] = trips_start["Profile Creation Timestamp"].dt.month
trips_start["Profile Creation Year"] = trips_start["Profile Creation Timestamp"].dt.year
trips_start["Profile Creation Hour"] = trips_start["Profile Creation Timestamp"].dt.hour
trips_start = trips_start.drop(columns=["Profile Creation Timestamp"])

df_train = df_train.join(trips_start.set_index('Id'), on='Id')

#Here, this is an totally optional step. I moved the 3 features added to the dataframe to the front of the dataframe to make it easy to see them when printing
#https://stackoverflow.com/questions/25122099/move-column-by-name-to-front-of-table-in-pandas
for col_name in ["Profile Creation Hour", "Profile Creation WeekDay", "Profile Creation Day", "Profile Creation Month", "Profile Creation Year"]:
  mid = df_train[col_name]
  df_train.drop(labels=col_name, axis=1, inplace = True)
  df_train.insert(10, col_name, mid)

df_train.drop(columns=["Profile Creation Timestamp"], inplace=True)

**Converting Categories to numeric**

In [6]:
cover_encoder = LabelEncoder()
df_train['Profile Cover Image Status'] = df_train['Profile Cover Image Status'].replace(np.nan, "unknown", regex=True)
df_train['Profile Cover Image Status'] = cover_encoder.fit_transform(df_train['Profile Cover Image Status'])
print(cover_encoder.classes_)

verification_encoder = LabelEncoder()
df_train['Profile Verification Status'] = verification_encoder.fit_transform(df_train['Profile Verification Status'])
print(verification_encoder.classes_)


'''
text_color_encoder = LabelEncoder()
df_train['Profile Text Color'] = df_train['Profile Text Color'].replace(np.nan, 'none', regex=True)
df_train['Profile Text Color'].apply(lambda x: x if(len(str(x)) == 6) else 'none')
df_train['Profile Text Color'] = text_color_encoder.fit_transform(df_train['Profile Text Color'])
print( text_color_encoder.classes_)

page_color_encoder = LabelEncoder()
df_train["Profile Page Color"] = df_train['Profile Page Color'].replace(np.nan, 'none', regex=True)
df_train['Profile Page Color'].apply(lambda x: x if(len(str(x)) == 6) else "")
df_train["Profile Page Color"] = page_color_encoder.fit_transform(df_train["Profile Page Color"])
print( page_color_encoder.classes_)

theme_color_encoder = LabelEncoder()
df_train["Profile Theme Color"] = df_train['Profile Theme Color'].replace(np.nan, 'none', regex=True)
df_train['Profile Theme Color'].apply(lambda x: x if(len(str(x)) == 6) else "")
df_train["Profile Theme Color"] = theme_color_encoder.fit_transform(df_train["Profile Theme Color"])
print( theme_color_encoder.classes_)
'''

location_encoder = LabelEncoder()
df_train["Location Public Visibility"] = df_train['Location Public Visibility'].replace(np.nan, '??', regex=True)
df_train['Location Public Visibility'] = df_train['Location Public Visibility'].str.lower()
df_train["Location Public Visibility"] = location_encoder.fit_transform(df_train["Location Public Visibility"])
print( location_encoder.classes_)

view_size_encoder = LabelEncoder()
df_train["Is Profile View Size Customized?"] = view_size_encoder.fit_transform(df_train["Is Profile View Size Customized?"])
print( view_size_encoder.classes_)

language_encoder = LabelEncoder()
df_train["User Language"] = df_train['User Language'].str.lower()
df_train["User Language"] = language_encoder.fit_transform(df_train["User Language"])
print( language_encoder.classes_)

category_encoder = LabelEncoder()
df_train["Profile Category"] = df_train['Profile Category'].replace(np.nan, 'unknown', regex=True)
df_train["Profile Category"] = df_train['Profile Category'].replace(' ', 'unknown', regex=True)
df_train['Profile Category'] = df_train['Profile Category'].str.lower()
df_train["Profile Category"] = category_encoder.fit_transform(df_train["Profile Category"])
print( category_encoder.classes_)

df_train["Profile Creation WeekDay"] = df_train["Profile Creation WeekDay"].map({'Monday':0, 'Tuesday':1, 'Wenesday':2, 'Thursday':3, "Friday":4, "Saturday":5, "Sunday":6})
in_order_weekdays = ["Monday", "Tuesday", "Wenesday", "Thursday", "Friday", "Saturday", "Sunday"]
print(in_order_weekdays)



['Not set' 'Set' 'unknown']
['Not verified' 'Pending' 'Verified']
['??' 'disabled' 'enabled']
[False  True]
['ar' 'ca' 'cs' 'de' 'en' 'en-gb' 'es' 'fi' 'fr' 'he' 'hu' 'id' 'it' 'ja'
 'ko' 'nl' 'no' 'pl' 'pt' 'ro' 'ru' 'sv' 'th' 'tr' 'zh-cn']
['business' 'celebrity' 'government' 'unknown']
['Monday', 'Tuesday', 'Wenesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


**Change unkown values in columns to NAN**


In [7]:
df_train["Profile Cover Image Status"].replace(2, np.nan, inplace=True)
'''
df_train["Profile Text Color"].replace(len(text_color_encoder.classes_)-1, np.nan, inplace=True)
df_train["Profile Page Color"].replace(len(page_color_encoder.classes_)-1, np.nan, inplace=True)
df_train["Profile Theme Color"].replace(len(theme_color_encoder.classes_)-1, np.nan, inplace=True)
'''
df_train["Location Public Visibility"].replace(0, np.nan, inplace=True)


**Rounding Daily Profile Visits and Clicks**

In [8]:
df_train["Avg Daily Profile Visit Duration in seconds"] = df_train["Avg Daily Profile Visit Duration in seconds"].round()
df_train["Avg Daily Profile Clicks"] = df_train["Avg Daily Profile Clicks"].round()

**Filling NAN values with appropriate replacement (Median)**

In [9]:
df_train = df_train.fillna(df_train.median())

**Normalizing continuous values using log10**

In [10]:
continuous_columns = ["Num of Followers", "Num of People Following", "Num of Status Updates", "Num of Direct Messages", "Avg Daily Profile Visit Duration in seconds", "Avg Daily Profile Clicks"]

for c in continuous_columns:
  df_train[c] = np.log10(df_train[c] + 1)

**Convert Columns to appropriate type**

In [11]:

df_train.astype({'Profile Cover Image Status': 'int64', 
                 'UTC Offset': 'int64',
                 'Location Public Visibility': 'int64',
                 'Profile Creation WeekDay': 'int64'
                 
                 })


,Id,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location Public Visibility,User Language,Profile Creation Year,Profile Creation Month,Profile Creation Day,Profile Creation WeekDay,Profile Creation Hour,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image
0,49I3SOKLI2CMNGP4,1,0,fa0a86,fc37c4,0a0101,0,-18000,2,4,2009,7,20,0,21,4.498710,3.332236,4.111498,2.672098,0,1.176091,0.602060,49I3SOKLI2CMNGP4.png
1,727IRIR59A3P88LK,0,0,ff0000,e6e6e6,001941,0,36000,2,4,2009,8,5,3,22,4.632032,2.419956,4.067406,3.256958,2,1.397940,0.602060,727IRIR59A3P88LK.png
2,LN95SD15SRPCEE8F,1,2,0d0101,000000,000000,0,-25200,2,4,2011,3,18,4,18,7.604635,2.594393,3.972943,4.772674,3,1.431364,1.113943,LN95SD15SRPCEE8F.png
3,TB11I7F0PN033D4T,1,2,0000ff,e0ff92,9ae4e8,0,-25200,2,4,2007,4,8,6,5,5.362043,2.846955,4.021520,3.430720,1,1.431364,0.477121,TB11I7F0PN033D4T.png
4,32PSGCK5PATHMR07,1,0,992f09,ddeef6,ffffff,0,-14400,2,4,2012,6,4,0,19,3.079181,3.230960,3.009876,0.954243,3,1.278754,0.698970,32PSGCK5PATHMR07.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,7SDJE48EFRPPNEJK,1,2,ffcc4d,ddeef6,080906,0,-18000,1,4,2009,10,22,3,15,6.476780,2.944483,3.862608,3.974512,3,1.301030,0.778151,7SDJE48EFRPPNEJK.png
2496,PRT8RDNG6E86518P,1,0,1200f0,78c9f2,000000,0,-14400,2,4,2008,7,15,1,20,4.407034,3.765966,4.237619,2.960471,2,1.146128,0.477121,PRT8RDNG6E86518P.png
2497,SRNIBIK27BQ2M3PB,0,0,0084b4,f6ffd1,9ae4e8,0,-10800,2,18,2011,7,21,3,13,5.574772,2.923762,4.718576,2.862131,0,1.361728,0.903090,SRNIBIK27BQ2M3PB.png
2498,6CP232J9R8N84702,1,0,69bf19,fff2c0,c1272c,0,-25200,1,4,2009,4,2,3,16,4.016072,2.850033,3.307282,2.501059,3,1.431364,0.778151,6CP232J9R8N84702.png


**Saving Model to CSV and Pkl**

In [12]:
df_train

,Id,Profile Cover Image Status,Profile Verification Status,Profile Text Color,Profile Page Color,Profile Theme Color,Is Profile View Size Customized?,UTC Offset,Location Public Visibility,User Language,Profile Creation Year,Profile Creation Month,Profile Creation Day,Profile Creation WeekDay,Profile Creation Hour,Num of Followers,Num of People Following,Num of Status Updates,Num of Direct Messages,Profile Category,Avg Daily Profile Visit Duration in seconds,Avg Daily Profile Clicks,Profile Image
0,49I3SOKLI2CMNGP4,1.0,0,fa0a86,fc37c4,0a0101,0,-18000.0,2.0,4,2009,7,20,0.0,21,4.498710,3.332236,4.111498,2.672098,0,1.176091,0.602060,49I3SOKLI2CMNGP4.png
1,727IRIR59A3P88LK,0.0,0,ff0000,e6e6e6,001941,0,36000.0,2.0,4,2009,8,5,3.0,22,4.632032,2.419956,4.067406,3.256958,2,1.397940,0.602060,727IRIR59A3P88LK.png
2,LN95SD15SRPCEE8F,1.0,2,0d0101,000000,000000,0,-25200.0,2.0,4,2011,3,18,4.0,18,7.604635,2.594393,3.972943,4.772674,3,1.431364,1.113943,LN95SD15SRPCEE8F.png
3,TB11I7F0PN033D4T,1.0,2,0000ff,e0ff92,9ae4e8,0,-25200.0,2.0,4,2007,4,8,6.0,5,5.362043,2.846955,4.021520,3.430720,1,1.431364,0.477121,TB11I7F0PN033D4T.png
4,32PSGCK5PATHMR07,1.0,0,992f09,ddeef6,ffffff,0,-14400.0,2.0,4,2012,6,4,0.0,19,3.079181,3.230960,3.009876,0.954243,3,1.278754,0.698970,32PSGCK5PATHMR07.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,7SDJE48EFRPPNEJK,1.0,2,ffcc4d,ddeef6,080906,0,-18000.0,1.0,4,2009,10,22,3.0,15,6.476780,2.944483,3.862608,3.974512,3,1.301030,0.778151,7SDJE48EFRPPNEJK.png
2496,PRT8RDNG6E86518P,1.0,0,1200f0,78c9f2,000000,0,-14400.0,2.0,4,2008,7,15,1.0,20,4.407034,3.765966,4.237619,2.960471,2,1.146128,0.477121,PRT8RDNG6E86518P.png
2497,SRNIBIK27BQ2M3PB,0.0,0,0084b4,f6ffd1,9ae4e8,0,-10800.0,2.0,18,2011,7,21,3.0,13,5.574772,2.923762,4.718576,2.862131,0,1.361728,0.903090,SRNIBIK27BQ2M3PB.png
2498,6CP232J9R8N84702,1.0,0,69bf19,fff2c0,c1272c,0,-25200.0,1.0,4,2009,4,2,3.0,16,4.016072,2.850033,3.307282,2.501059,3,1.431364,0.778151,6CP232J9R8N84702.png


In [13]:
df_train.to_pickle("test_cleaned.pkl")
df_train.to_csv("test_cleaned.csv")